In [67]:
import os
import cv2
import numpy as np
import sys
import glob
import json
import h5py
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import pretrainedmodels
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(pretrainedmodels.model_names)
model_name = 'inceptionresnetv2' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model = model
model.eval()
print(pretrainedmodels.pretrained_settings[model_name])

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']
{'imagenet': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth', 'input_space': 'RGB', 'input_size': [3, 299, 299], 'input_range': [0, 1], 'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5], 'num_classes': 1000}, 'imagenet+background': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth', 'input_space': 'RGB', 

In [3]:
dset_path = os.path.join('/'.join(os.getcwd().split('/')[:-1]),'Datasets')
msvd_path = os.path.join(os.path.join(dset_path,'MSVD'),'YouTube')
msrvtt_trnval_path = os.path.join(os.path.join(dset_path,'MSR-VTT'),'TrainValVideo')
msrvtt_test_path = os.path.join(os.path.join(dset_path,'MSR-VTT'),'TestVideo')

msvd_name_list = glob.glob(msvd_path+'/*')

url2id = {}
for line in open('youtube_mapping.txt','r').readlines():
    url2id[line.strip().split(' ')[0]] = line.strip().split(' ')[-1]
    
    
    
def file_to_dict(path):
    dic = dict()
    lst = []
    fil = open(path,'r+')
    img =[]
    for f in fil.readlines():
        l = f.split() 
        ll = ' '.join(x for x in l[1:])
        if l[0] not in dic:
            dic[l[0]] = [ll]
        else:
            dic[l[0]].append(ll)

    return dic

train_dict = file_to_dict('MSVD/captions/sents_train_lc_nopunc.txt')
val_dict = file_to_dict('MSVD/captions/sents_val_lc_nopunc.txt')
test_dict = file_to_dict('MSVD/captions/sents_test_lc_nopunc.txt')

train_name_list = list(train_dict.keys())
val_name_list = list(val_dict.keys())
test_name_list = list(test_dict.keys())

In [4]:
data_transform = transforms.Compose([transforms.ToPILImage(),
                                     transforms.Resize((800,600)),
                                     transforms.ToTensor()])
def video2tensor(video_path):
    vidObj = cv2.VideoCapture(video_path) 
    count = 0
    success = 1
    frames = []
    fail = 0
    while success: 
        # OpenCV Uses BGR Colormap
        success, image = vidObj.read() 
        if success:
            RGBimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #transform images
            #print(RGBimage.shape)
            frames.append(data_transform(RGBimage))
            count += 1
        else:
            fail += 1
    vidObj.release()
    frames = torch.stack(frames)
    #frames = torch.from_numpy(frames)
    # take 28 frames per clip uniformly
    interval = count//28
    frames = frames[range(0,interval*28,interval)]
    return frames,count,fail

In [ ]:
# with torch.no_grad():
#     with h5py.File('MSVD_APPEARANCE_INCEPTIONRESNETV2.hdf5', 'w') as f:
#         for name in tqdm(msvd_name_list):
#             tensor,_,_ = video2tensor(name)
#             output_features = model.features(tensor).mean(dim=(2,3))
#             ide = url2id[name.split('\\')[-1].split('.')[0]]
#             f.create_dataset(ide, data = output_features.numpy())

In [5]:
tensor,_,_ = video2tensor(msvd_name_list[0])

In [6]:
tensor.size()

torch.Size([28, 3, 800, 600])

# Object detection and feature extration

In [7]:
import torchvision.transforms as T
import requests
from PIL import Image,ImageDraw

model = torch.hub.load('facebookresearch/detr','detr_resnet101',pretrained=True)

# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

N = 5  # number of objects to take.

Using cache found in /home/nasibullah/.cache/torch/hub/facebookresearch_detr_master


In [ ]:
with torch.no_grad():
    with h5py.File('MSVD__RESNEXT101.hdf5', 'w') as f:
        for name in tqdm(msvd_name_list):
            _,tensor,_,_ = video2tensor(name)
            output_features = model(tensor.to(device)).mean(dim=(2,3,4))
            ide = url2id[name.split('/')[-1].split('.')[0]]
            f.create_dataset(ide, data = output_features.cpu().numpy())

In [38]:
data_reverse_transform = transforms.Compose([transforms.ToPILImage()])
data_transform2 = transforms.Compose([transforms.Resize((299,299)),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
@torch.no_grad()
def tensor2objectfeat(tensor,N):
    object_boxes = []
    object_labels = []
    object_prob = []
    object_feature = []
    frame_index = []
    object_frames = []
    i = -1
    for ten in tensor:
        i += 1
        output = model(ten.unsqueeze(0))
        #print('logits :',output['pred_logits'].size())
        #print('boxes :',output['pred_boxes'].size())
        prob, scores = output['pred_logits'].squeeze(0).topk(1)
        boxes = output['pred_boxes'].squeeze(0)
        #print('prob :',prob.size())
        #print('scores :',scores.size())
        for s,b,p in zip(scores,boxes,prob):
            cls = s
            if cls >= len(CLASSES):
                continue
            label = CLASSES[cls]
            frame_index.append(i)
            box = b.cpu()*torch.tensor([800,600,800,600])
            object_labels.append(label)
            x,y,w,h = box
            x0,x1 = x - w//2, x+ w//2
            y0,y1 = y-h//2, y+h//2
            object_boxes.append(torch.tensor([x0,y0,x1,y1]))
            object_prob.append(p)
            
    object_boxes,object_prob = torch.stack(object_boxes), torch.stack(object_prob)
    #print('object label before :',object_labels)
    #print('Onject scores before : ',object_prob)
    unique_frame_index = np.array(frame_index)
    
    #Finding unique elements
    unique_object_labels,indexes = np.unique(object_labels,return_index=True)
    unique_object_prob = object_prob.cpu().numpy()[indexes]
    unique_object_boxes = object_boxes.cpu().numpy()[indexes]
    unique_frame_index = unique_frame_index[indexes]
    
    if len(unique_object_labels) >5 :
        unique_object_prob,p_ind = torch.tensor(unique_object_prob).squeeze(1).topk(N)
        unique_object_labels = [unique_object_labels[j] for j in p_ind]
        unique_frame_index = [unique_frame_index[kk] for kk in p_ind]
        unique_object_boxes = unique_object_boxes[p_ind]
        
#     print('Object label after :',object_labels)
#     print('Object scores after :',object_prob)
    
    for b in range(len(unique_object_labels)):
        object_frames.append(data_reverse_transform(tensor[unique_frame_index[b]]))

        
    return unique_object_boxes, unique_object_labels,unique_object_prob,object_frames

        

In [97]:
idx = 6
tensor,_,_ = video2tensor(msvd_name_list[idx])
a,b,c,d = tensor2objectfeat(tensor,N)
raw_name = msvd_name_list[idx].split('/')[-1].split('.')[0]
uid = url2id[raw_name]

if uid in train_dict.keys(): 
    text = train_dict[uid]
elif uid in val_dict.keys():
    text = val_dict[uid]
else:
    text = test_dict[uid]
    
text

['a woman is doing a dance on a rooftop',
 'a girl danced on the roof of a building',
 'a girl is dancing on a decorated floor',
 'a girl is dancing',
 'a girl is dancing',
 'a lady danced on a rooftop',
 'a woman dances on a decorated brick floor',
 'a woman in a martial arts costume is dancing',
 'a woman is dancing on the terrace of a building and stepping on colored powder patterns on the floor',
 'a woman is dancing',
 'a woman is dancing',
 'a woman is performing a dance on a floor that is covered with colored geometric shapes',
 'a woman is ritually dancing',
 'the woman is dancing',
 'kavithai kelungal song from punnagai mannan',
 'a girl dacing on the house',
 'a girl dancing on roof',
 'a lady is dancing',
 'a woman dances on a rooftop',
 'a woman is dancing',
 'a women dancing',
 'actor revathi dancing in punnagai mannan for the song kavithai kelungal',
 'the clouds are behind the mountain landscapes',
 'the clouds behind the mountains are beautiful',
 'a floor decoration on

In [98]:
b,c

(['umbrella', 'person', 'horse', 'handbag', 'surfboard'],
 tensor([10.5621, 10.4308,  9.5068,  9.2877,  8.6618]))

In [102]:
i = 4
draw = ImageDraw.Draw(d[i])
draw.rectangle(tuple(a[i]))
d[i].show()

In [ ]:
if len(object_labels)>5:
    object_labels,ind = np.unique(object_labels,return_index=True)
    object_boxes,object_prob = object_boxes[ind], object_prob[ind]
    frame_index = [frame_index[jj] for jj in ind]

In [ ]:
a = ['cat','man','person','vessel','person']
aa =np.array( [0.23,0.54,0.45,0.34,0.2])
b,c = np.unique(a,return_index=True)
list(b),c

In [ ]:
aa[c]